In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install seqeval

In [ ]:
from transformers import BertTokenizerFast, BertForPreTraining,BertForTokenClassification ,Trainer, TrainingArguments
import os
import torch 
import pandas as pd
import datasets

In [ ]:
torch.cuda.is_available()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("german_legal_entity_recognition","all",ignore_verifications =True , download_mode="force_redownload")

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [ ]:
(dataset["train"].features["ner_tags"].feature)

In [ ]:
label_names = dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
len(label_names) 

In [ ]:
#Load Model and Tokenizer
from transformers import AutoTokenizer

tokenizer = BertTokenizerFast.from_pretrained('deepset/gbert-large')
model = BertForTokenClassification.from_pretrained('deepset/gbert-large', num_labels=len(label_names) )

In [ ]:
def tokenize_and_align_labels(examples):
    #print(examples.keys())
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_metric
import numpy as np
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset,compute_metrics=compute_metrics, data_collator=data_collator,
    tokenizer=tokenizer)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model.save_pretrained('./Legal_NER_2/')

In [ ]:
tokenizer.save_pretrained('./Legal_TOK_2/')